### Ejemplo de uso de Spark Streaming con Kafka y direct Stream.

In [1]:
# imports necesarios
# Spark Streaming
from pyspark.streaming import StreamingContext  
# Kafka
from pyspark.streaming.kafka import KafkaUtils
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.functions import split

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import json
import string
import re

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/josemanuel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### Funciones necesarias para la prueba.

In [2]:
# función para sacar el texto de cada tweet
def returnText(x):
    try:
        return x['text']
    except:
        return ""


# función que se le pasa el texto del tweet y las palabras a eliminar del mismo (stopwords) porque no tengan
# relevancia en el lenguaje para hacer el análisis de sentimiento.
def eliminar_stopwords(texto, palabras_eliminar):
    tok = nltk.tokenize
    palabras = tok.word_tokenize(texto)
        
    palabras_salida = []
        
    for palabra in palabras:
        if palabra not in palabras_eliminar:
            palabras_salida.append(palabra)
        
    salida = ""
    for i in range(len(palabras_salida)):
        if palabras_salida[i] in string.punctuation:
            salida = salida.strip()+palabras_salida[i] + " "
        else:
            salida += palabras_salida[i] + " "

    return salida


# función para reemplazar/eliminar emojis
def handle_emojis(tweet):
    emoji_pattern = re.compile(u'['u'\U0001F300-\U0001F64F'u'\U0001F680-\U0001F6FF'u'\
                               \u2600-\u26FF\u2700-\u27BF]+', re.UNICODE)
 
    tweet = emoji_pattern.sub(r' ', tweet)

    return tweet


# función para realizar toda la limpieza y preprocesado de los textos de los tweets
def preprocesado_texto(tweet):
    processed_tweet = []
 
    # remover todos los espacios al principio y al final del tweet
    tweet = re.sub('  +', ' ', tweet).strip()
    # remover las urls
    tweet = re.sub(r'http\S+', '', tweet)
    # remover los números
    translator = str.maketrans(dict.fromkeys("0123456789"))
    tweet = tweet.translate(translator)    
    # remover los @usuario
    tweet = re.sub(r'@[\S]+', ' ', tweet)
    # remover los #hashtags
    tweet = re.sub(r'#(\S+)', r' \1 ', tweet)
    # remover RT (retweet)
    tweet = re.sub(r'\bRT\b', '', tweet)
    # reemplazar .. o más con un espacio
    tweet = re.sub(r'\.{2,}', ' ', tweet)
    # remover espacios sobrantes
    tweet = tweet.strip('"\'')
    # reemplazar múltiples espacios con un sólo espacio
    tweet = re.sub(r'\s+', ' ', tweet)
    # pasar todo a minúscula
    tweet = tweet.lower()
    # remover signos de puntuación
    translator = str.maketrans(dict.fromkeys(string.punctuation + "'\"¿?¡!,.():;´’-"))
    tweet = tweet.translate(translator)
    # convertir la repetición de una letra más de 2 veces a 1
    # biennnnn --> bien
    tweet = re.sub(r'(.)\1+', r'\1\1', tweet)
    # remover - & '
    tweet = re.sub(r'(-|\')', '', tweet) 
    # eliminar acentos
    tweet = tweet.translate(str.maketrans('áàéèíìóòúùü','aaeeiioouuu'))
    # reemplazar emojis
    tweet = handle_emojis(tweet)
    tweet = re.sub("[^A-Za-z0-9]+$",'',tweet)
    tweet = re.sub("^[^A-Za-z0-9]+",'',tweet)
    tweet = re.sub("[\$*&!?///\º\'\’\‘\|()%/\"{}@;:+\[\]\–\”\…\“\】\【=]",'',tweet)

    # remover stopwords
    spanish_stopwords = stopwords.words('spanish')
    tweet = eliminar_stopwords(tweet, spanish_stopwords)
    
    words = tweet.split()

    for word in words:
        processed_tweet.append(word)        
            
    return ' '.join(processed_tweet)   

#### Creamos una función donde obtenemos los datos desde Kafka y mostramos por pantalla cada x tiempo algo de información de los datos obtenidos en cada batch por Spark Streaming.

In [3]:
def createContext():
    conf = (SparkSession\
          .builder\
          .appName("twitter")\
          .master("spark://MacBook-Pro-de-Jose.local:7077")\
          .config("spark.io.compression.codec", "snappy")\
          .getOrCreate())
    
    sc = SparkContext.getOrCreate(conf=conf)
    sc.setLogLevel("WARN")
 
    ssc = StreamingContext(sc, 30)
    ssc.checkpoint("checkpoint")
    
    # topic del que recoger los datos en español
    topic = "TopicSpanish"
    # conexión con kafka
    fromOffset=None
    opts = { "metadata.broker.list": "localhost:9092",
         "auto.offset.reset" : 'largest' }
    directKafkaStream = KafkaUtils.createDirectStream( ssc, [topic], opts, fromOffsets=fromOffset )

    # extraer tweets cogiendo directamente el texto que será el atributo a analizar
    tweets_texto = directKafkaStream.map(lambda v: json.loads(v[1])).map(returnText)
    
    # número de tweets por batch
    count_batch = directKafkaStream.count().map(lambda x:('Tweets del batch: %s' % x))

    # recuento por ventana de tiempo
    count_window = directKafkaStream.countByWindow(30,30)\
        .map(lambda x:('Tweets total (medio minuto rolling): %s' % x))
    
    # obtenemos cada texto dividido en palabras y le hacemos las transformaciones necesarias 
    # para eliminar @usuario, RT, urls, y palabras que no son relevantes para el análisis
    texto_procesado = tweets_texto.map(lambda row: (preprocesado_texto(row)))
    tokens = texto_procesado.map(lambda row: (word_tokenize(row)))
    
    # palabras más repetidas
    words = texto_procesado.flatMap(lambda tweet:tweet.split(" ")).countByValue()\
        .transform(lambda rdd:rdd.sortBy(lambda x:-x[1]))
    
    # sacar por pantalla todos los procesamientos realizados
    count_batch.pprint(5)
    count_window.pprint(5)
    tweets_texto.pprint(5)
    texto_procesado.pprint(5)
    tokens.pprint(5)
    words.pprint(5)
    
    return ssc

In [5]:
# lanzamos el proceso en streaming para recoger los tweets y hacerles el preprocesado y ciertos cálculos
ssc = createContext()
ssc.start()
ssc.awaitTerminationOrTimeout(120)
ssc.stop(stopGraceFully = True)

print("!!!!!!!!!!!!!!!!!!!!STOPPED!!!!!!!!!!!!!!!!!!!!")

-------------------------------------------
Time: 2019-08-15 15:03:30
-------------------------------------------
Tweets del batch: 78

-------------------------------------------
Time: 2019-08-15 15:03:30
-------------------------------------------
Tweets total (medio minuto rolling): 78

-------------------------------------------
Time: 2019-08-15 15:03:30
-------------------------------------------
Me tienen q poner la vacuna de los 16 lpm
RT @escorpio_hn: Los #Escorpio suelen ser personas alternativas, se suelen alejar de modas o de cosas que hace todo el mundo. Les gusta ser…
RT @laraizaurieta: Mi ex se burla de mi porque trabajo en Mc Donalds

Gano más que el
Vivo sola
Mantengo como puedo a mi bebé 
No dependo d…
RT @2RADl0: #AnabelQuevedo #14Ago #53Dias

¿ Dónde está Anabel Quevedo ?  

https://t.co/gz5fCli3Fi
@LeyvaJulia @imagenZea @ImagenTVMex Igualmente Julia! Excelente día.
...

-------------------------------------------
Time: 2019-08-15 15:03:30
---------------------------

-------------------------------------------
Time: 2019-08-15 15:05:30
-------------------------------------------
elecciones tn mostraron cuales causas judiciales afectan macri funcionarios
menuda maravilla union berlin debuta bundesliga celebra primer partido casa imprimendo foto
bop convocatoria provision plaza tecnicao medio opem pagina
visperas cumpleaños queremos compartir ustedes gran regalo presentamos nuest
cuanto pagan provocar dando hecho paso pequeña empr
...

-------------------------------------------
Time: 2019-08-15 15:05:30
-------------------------------------------
['elecciones', 'tn', 'mostraron', 'cuales', 'causas', 'judiciales', 'afectan', 'macri', 'funcionarios']
['menuda', 'maravilla', 'union', 'berlin', 'debuta', 'bundesliga', 'celebra', 'primer', 'partido', 'casa', 'imprimendo', 'foto']
['bop', 'convocatoria', 'provision', 'plaza', 'tecnicao', 'medio', 'opem', 'pagina']
['visperas', 'cumpleaños', 'queremos', 'compartir', 'ustedes', 'gran', 'regalo', 'presentamo

Con esta primera prueba de uso de Spark Streaming, conectamos con Kafka recogiendo los datos de twitter que están llegando al sistema, y hacemos unas primeras pruebas de preprocesado de los textos, y les aplicamos ciertos análisis o funciones como ver en cada batch las palabrás más repetidas o sacar el recuento de tweets de cada batch procesado.

Vamos a seguir avanzando en el streaming por el uso de Structured Streaming, que es más moderno y nos permite usar los datos recogidos en streaming como dataframes, lo cual está más alineado con el resto de trabajo realizado ya que estamos trabajando con dataframes a lo largo de todo el desarrollo realizado.